<CENTER><img src="../../images/ATLASOD.gif" style="width:50%"></CENTER>

# How to rediscover the Higgs boson yourself!
This notebook uses ATLAS Open Data http://opendata.atlas.cern to show you the steps to rediscover the Higgs boson yourself!

ATLAS Open Data provides open access to proton-proton collision data at the LHC for educational purposes. ATLAS Open Data resources are ideal for high-school, undergraduate and postgraduate students.

Notebooks are web applications that allow you to create and share documents that can contain for example:
1. live code
2. visualisations
3. narrative text

This analysis loosely follows the discovery of the Higgs boson by ATLAS https://arxiv.org/pdf/1207.7214.pdf (mostly Section 5 and 5.1)

By the end of this notebook you will be able to:
1. rediscover the Higgs boson yourself!
2. know some general principles of a particle physics analysis

Feynman diagram pictures are borrowed from our friends at https://www.particlezoo.net

<CENTER><img src="images/feynman_diagrams/Hyy_feynman.png" style="width:40%"></CENTER>

<a id='contents'></a>

Contents: 

[Running a Jupyter notebook](#running) <br />
[First time setup on your computer (no need on mybinder)](#setup_computer) <br />
[To setup everytime](#setup_everytime) <br />
[Lumi, fraction, file path](#fraction) <br />
[Samples](#samples) <br />
[Changing a cut](#changing_cut) <br />
[Applying a cut](#applying_cut) <br />
[Plotting](#plotting) <br />
[What can you do to explore this analysis?](#going_further) <br />

<a id='running'></a>

## Running a Jupyter notebook

To run the whole Jupyter notebook, in the top menu click Cell -> Run All.

To propagate a change you've made to a piece of code, click Cell -> Run All Below.

You can also run a single code cell, by clicking Cell -> Run Cells, or using the keyboard shortcut Shift+Enter.

<a id='setup_computer'></a>

## First time setup on your computer (no need on mybinder)
This first cell only needs to be run the first time you open this notebook on your computer. 

If you close Jupyter and re-open on the same computer, you won't need to run this first cell again.

If you open on binder, you don't need to run this cell.

In [126]:
#import sys
#!{sys.executable} -m pip install --upgrade --user pip # update the pip package installer
#!{sys.executable} -m pip install -U numpy==2.0.0 pandas==2.2.2 uproot==5.3.9 matplotlib==3.9.0 lmfit==1.3.1 awkward-pandas==2023.8.0 aiohttp==3.9.5 requests==2.32.3 --user # install required packages

[Back to contents](#contents)

<a id='setup_everytime'></a>

## To setup everytime
Cell -> Run All Below

to be done every time you re-open this notebook

We're going to be using a number of tools to help us:
* uproot: lets us read .root files typically used in particle physics into data formats used in python
* pandas: lets us store data as dataframes, a format widely used in python
* numpy: provides numerical calculations such as histogramming
* matplotlib: common tool for making plots, figures, images, visualisations
* lmfit: tool for statistical fitting

In [127]:
#!pip install lmfit

In [128]:
#!pip install awkward-pandas

In [129]:
import uproot # for reading .root files
import awkward as ak 
import time # to measure time to analyse
import math # for mathematical functions such as square root
import numpy as np # # for numerical calculations such as histogramming
import matplotlib.pyplot as plt # for plotting
from matplotlib.ticker import MaxNLocator,AutoMinorLocator # for minor ticks;
from lmfit.models import PolynomialModel, GaussianModel # for the signal and background fits
import requests # for HTTP access
import aiohttp # HTTP client support
import vector

### check tree content of old and new

In [130]:
# new file
pname = '/project/etp1/dkoch/ATLASOpenData/ntuples-data-samples/data15_allyear.root'
pname = '/project/etp1/dkoch/ATLASOpenData/ntuples-data-samples/data16_allyear_G.root'
tree = uproot.open(pname+':analysis')

print(tree.num_entries)

tree.keys()

108492747


['ScaleFactor_PILEUP',
 'mcWeight',
 'xsec',
 'trigE',
 'trigM',
 'ScaleFactor_BTAG',
 'jet_n',
 'jet_pt',
 'jet_eta',
 'jet_phi',
 'jet_e',
 'jet_DL1d77_isBtagged',
 'jet_jvt',
 'largeRJet_n',
 'largeRJet_pt',
 'largeRJet_eta',
 'largeRJet_phi',
 'largeRJet_e',
 'largeRJet_m',
 'largeRJet_D2',
 'ScaleFactor_ELE',
 'ScaleFactor_MUON',
 'lep_n',
 'lep_type',
 'lep_pt',
 'lep_eta',
 'lep_phi',
 'lep_e',
 'lep_charge',
 'lep_ptvarcone30',
 'lep_topoetcone20',
 'lep_z0',
 'lep_d0',
 'lep_d0sig',
 'lep_isTight',
 'lep_isTightID',
 'lep_isTightIso',
 'ScaleFactor_PHOTON',
 'photon_n',
 'photon_pt',
 'photon_eta',
 'photon_phi',
 'photon_e',
 'photon_ptcone20',
 'photon_topoetcone40',
 'photon_isTight',
 'photon_isTightID',
 'photon_isTightIso',
 'ScaleFactor_TAU',
 'tau_n',
 'tau_pt',
 'tau_eta',
 'tau_phi',
 'tau_e',
 'tau_charge',
 'tau_nTracks',
 'tau_isTight',
 'tau_RNNJetScore',
 'tau_RNNEleScore',
 'met',
 'met_phi',
 'met_mpx',
 'met_mpy']

In [5]:
import uproot

# Define the path to the directory and the list of samples
tuple_path = "/project/etp1/dkoch/ATLASOpenData/ntuples-data-samples/"
tuple_path = "/project/etp1/dkoch/ATLASOpenData/GamGam/Data/"



samples_list = ['data15_allyear', 'data16_allyear_A', 'data16_allyear_B', 
                'data16_allyear_C', 'data16_allyear_D', 'data16_allyear_E', 
                'data16_allyear_F', 'data16_allyear_G', 'data16_allyear_H']


samples_list = [
    'data15_periodD', 'data15_periodG', 'data16_periodA', 'data16_periodD', 'data16_periodG', 'data16_periodL',
    'data15_periodE', 'data15_periodH', 'data16_periodB', 'data16_periodE', 'data16_PeriodI',
    'data15_periodF', 'data15_periodJ', 'data16_periodC', 'data16_periodF', 'data16_periodK'
]



print(samples_list)



# Initialize a variable to store the total number of entries
total_entries = 0

# Iterate through each sample in the list
for sample in samples_list:
    # Construct the full path to the .root file
    pname = f"{tuple_path}{sample}.root"
    
    try:
        # Open the .root file and access the 'analysis' tree
        with uproot.open(pname + ':analysis') as tree:
            # Add the number of entries in the current tree to the total
            total_entries += tree.num_entries
    except Exception as e:
        print(f"Error processing {pname}: {e}")

# Print the total number of entries across all samples
print(f"Total number of entries: {total_entries}")


['data15_periodD', 'data15_periodG', 'data16_periodA', 'data16_periodD', 'data16_periodG', 'data16_periodL', 'data15_periodE', 'data15_periodH', 'data16_periodB', 'data16_periodE', 'data16_PeriodI', 'data15_periodF', 'data15_periodJ', 'data16_periodC', 'data16_periodF', 'data16_periodK']
Total number of entries: 4998873


In [6]:
myt = tree.iterate(["photon_n","photon_pt","photon_eta","photon_phi","photon_e",
                            "photon_isTightID","photon_ptcone20"], # add more variables here if you want to use them
                           library="ak")

In [7]:
data = next(myt)

In [8]:
data = data.query('photon_n>=2')

AttributeError: no field named 'query'

In [11]:
data.photon_e
data["photon_isTightID"]

<Array [[False, False, False], ..., [False, ...]] type='289333 * var * bool'>

[Back to contents](#contents)

<a id='fraction'></a>

## Lumi, fraction, file path

General definitions of luminosity, fraction of data used, where to access the input files

In [15]:
#lumi = 0.5 # fb-1 # data_A only
#lumi = 1.9 # fb-1 # data_B only
#lumi = 2.9 # fb-1 # data_C only
#lumi = 4.7 # fb-1 # data_D only
lumi = 10 # fb-1 # data_A,data_B,data_C,data_D
MeV = 0.001
GeV = 1
fraction = 1 # reduce this is you want the code to run quicker

#tuple_path = "Input/GamGam/Data/" # local 
tuple_path = "/project/etp1/dkoch/ATLASOpenData/ntuples-data-samples/"
tuple_path = "/project/etp1/dkoch/ATLASOpenData/GamGam/Data/"
#tuple_path = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/GamGam/Data/" # web address

<a id='samples'></a>

## Samples

Samples to process

In [16]:
#samples_list = ['data_A','data_B','data_C','data_D'] # add if you want more data
samples_list = ['data15_allyear', 'data16_allyear_A', 'data16_allyear_B', 
                'data16_allyear_C', 'data16_allyear_D', 'data16_allyear_E', 
                'data16_allyear_F', 'data16_allyear_G', 'data16_allyear_H' ]
#samples_list = ['data15_allyear']  
samples_list = [
    'data15_periodD', 'data15_periodG', 'data16_periodA', 'data16_periodD', 'data16_periodG', 'data16_periodL',
    'data15_periodE', 'data15_periodH', 'data16_periodB', 'data16_periodE', 'data16_PeriodI',
    'data15_periodF', 'data15_periodJ', 'data16_periodC', 'data16_periodF', 'data16_periodK'
]

[Back to contents](#contents)

Define function to get data from files

The datasets used in this notebook have already been filtered to include at least 2 photons per event, so that processing is quicker.

In [17]:
def get_data_from_files():


    frames = [] # define empty list to hold data
    for val in samples_list: # loop over each file
        fileString = tuple_path+val+".root" # file name to open
        temp = read_file(fileString,val) # call the function read_file defined below
        frames.append(temp) # append dataframe returned from read_file to list of dataframes
    data = ak.concatenate(frames) # concatenatelist of awkward arrays
    
    return data # return dataframe

Define function to calculate diphoton invariant mass

In [18]:
def calc_myy(photon_pt,photon_eta,photon_phi,photon_e):
    # construct awkward 4-vector array
    p4 = vector.awk(ak.zip(dict(pt=photon_pt, eta=photon_eta, phi=photon_phi, E=photon_e)))
    # calculate invariant mass of first 4 leptons
    # [:, i] selects the i-th lepton in each event
    # .M calculates the invariant mass
    return (p4[:, 0] + p4[:, 1]).M * MeV



<a id='changing_cut'></a>

In [19]:
# Cut on photon reconstruction quality
# paper: "Photon candidates are required to pass identification criteria"
def cut_photon_reconstruction(photon_isTightID):
    # Keep only the events where the first two photons are both True
    return (photon_isTightID[:, 0] == True) & ak.any(photon_isTightID[:, 1] == True)

    
# Cut on Transverse momentum
# paper: "The leading (sub-leading) photon candidate is required to have ET > 40 GeV (30 GeV)"
def cut_photon_pt(photon_pt):

    return (photon_pt[:,0] >40 ) & (photon_pt[:,1]>30 )

# Cut on energy isolation
# paper: "Photon candidates are required to have an isolation transverse energy of less than 4 GeV"
def cut_isolation_pt(photon_ptcone20):
# want to keep events where isolation eT<4000 MeV
    return (photon_ptcone20[:,0]<4) & (photon_ptcone20[:,1]<4 )

# Cut on pseudorapidity in barrel/end-cap transition region
# paper: "excluding the calorimeter barrel/end-cap transition region 1.37 < |η| < 1.52"
def cut_photon_eta_transition(photon_eta):
# want to keep events where modulus of photon_eta is outside the range 1.37 to 1.52
    return ((np.abs(photon_eta[:,0])>1.52) | (np.abs(photon_eta[:,0])<1.37)) & ((np.abs(photon_eta[:,1])>1.52) | (np.abs(photon_eta[:,1])<1.37))


def cut_n_photon(photon_n):
# >=2
    return photon_n>=2


[Back to contents](#contents)

<a id='applying_cut'></a>

## Applying a cut 

If you add a cut: Cell -> Run All Below

In [20]:
import time
import numpy as np
import pandas as pd
import uproot
import awkward as ak

# Initialize global cutflow dictionary
global_cutflow = {
    "Before Cuts": 0,
    "After photon reconstruction cut": 0,
    "After photon pt cut": 0,
    "After photon isolation cut": 0,
    "After photon eta transition cut": 0
}

def read_file(path, sample, fraction=1.0):
    start = time.time()
    print(f"\tProcessing: {sample}")
    data_all = []

    # Initialize local cutflow dictionary
    local_cutflow = {
        "Before Cuts": 0,
        "After photon reconstruction cut": 0,
        "After photon pt cut": 0,
        "After photon isolation cut": 0,
        "After photon eta transition cut": 0
    }

    with uproot.open(path + ":analysis") as tree:
        numevents = tree.num_entries
        for data in tree.iterate(["photon_pt", "photon_eta", "photon_phi", "photon_e",
                                  "photon_isTightID", "photon_ptcone20"],
                                 library="ak",
                                 entry_stop=int(numevents*fraction)):
            
            local_cutflow["Before Cuts"] += len(data)
            global_cutflow["Before Cuts"] += len(data)

            data = data[cut_photon_reconstruction(data.photon_isTightID)]
            local_cutflow["After photon reconstruction cut"] += len(data)
            global_cutflow["After photon reconstruction cut"] += len(data)

            data = data[cut_photon_pt(data.photon_pt)]
            local_cutflow["After photon pt cut"] += len(data)
            global_cutflow["After photon pt cut"] += len(data)

            data = data[cut_isolation_pt(data.photon_ptcone20)]
            local_cutflow["After photon isolation cut"] += len(data)
            global_cutflow["After photon isolation cut"] += len(data)

            data = data[cut_photon_eta_transition(data.photon_eta)]
            local_cutflow["After photon eta transition cut"] += len(data)
            global_cutflow["After photon eta transition cut"] += len(data)

            data['myy'] = calc_myy(data.photon_pt, data.photon_eta, data.photon_phi, data.photon_e)
            data_all.append(data)

            elapsed = time.time() - start
            print(f"\t\t nIn: {local_cutflow['Before Cuts']},\t nOut: \t{len(data)}\t in {round(elapsed,1)}s")

    final_data = ak.concatenate(data_all, highlevel=True)

    print("\nLocal Cutflow Summary:")
    print(f"{'Cut':<35}{'Entries':<15}{'Percentage Reduction':<25}")
    print("-" * 75)
    previous_entries = local_cutflow["Before Cuts"]
    for cut, entries in local_cutflow.items():
        if cut == "Before Cuts":
            print(f"{cut:<35}{entries:<15}-")
        else:
            reduction = (previous_entries - entries) / previous_entries * 100
            print(f"{cut:<35}{entries:<15}{reduction:.2f}%")
        previous_entries = entries
    total_reduction = (local_cutflow["Before Cuts"] - local_cutflow["After photon eta transition cut"]) / local_cutflow["Before Cuts"] * 100
    print("-" * 75)
    print(f"{'Total entries after all cuts':<35}{local_cutflow['After photon eta transition cut']:<15}{total_reduction:.2f}%")

    return final_data

def print_global_cutflow():
    print("\nGlobal Cutflow Summary:")
    print(f"{'Cut':<35}{'Entries':<15}{'Percentage Reduction':<25}")
    print("-" * 75)
    previous_entries = global_cutflow["Before Cuts"]
    for cut, entries in global_cutflow.items():
        if cut == "Before Cuts":
            print(f"{cut:<35}{entries:<15}-")
        else:
            reduction = (previous_entries - entries) / previous_entries * 100
            print(f"{cut:<35}{entries:<15}{reduction:.2f}%")
        previous_entries = entries
    total_reduction = (global_cutflow["Before Cuts"] - global_cutflow["After photon eta transition cut"]) / global_cutflow["Before Cuts"] * 100
    print("-" * 75)
    print(f"{'Total entries after all cuts':<35}{global_cutflow['After photon eta transition cut']:<15}{total_reduction:.2f}%")


[Back to contents](#contents)

This is where the processing happens (this will take some minutes)

In [21]:
#pname = '/project/etp1/dkoch/ATLASOpenData/ntuples-data-samples/data15_allyear.root'
start = time.time() # time at start of whole processing
data = get_data_from_files() # process all files
#data = read_file_new(pname) #
elapsed = time.time() - start # time after whole processing
print("Time taken: "+str(round(elapsed,1))+"s") # print total time taken to process every file
print_global_cutflow()

	Processing: data15_periodD


NameError: name 'vector' is not defined


Global Cutflow Summary:
Cut                                Entries        Percentage Reduction     
---------------------------------------------------------------------------
Before Cuts                        4998873        -
After photon reconstruction cut    488229         90.23%
After photon pt cut                259806         46.79%
After photon isolation cut         157854         39.24%
After photon eta transition cut    156974         0.56%
---------------------------------------------------------------------------
Total entries after all cuts       156974         96.86%


<a id='plotting'></a>

## Plotting
If you only want a make a change in the plot: Cell -> Run All Below

Define function to plot the data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from awkward import Array, to_list
from lmfit.models import PolynomialModel, GaussianModel
from matplotlib.ticker import AutoMinorLocator, MaxNLocator

def plot_data(data):
    # Ensure data is an awkward array
    myy_data = Array(data['myy']) if not isinstance(data['myy'], Array) else data['myy']
    
    # Check for NaN values
    myy_np = to_list(myy_data)
    if np.any(np.isnan(myy_np)):
        raise ValueError("Input data contains NaN values!")

    xmin = 100  # GeV
    xmax = 160  # GeV
    step_size = 2  # GeV
    
    bin_edges = np.arange(start=xmin, stop=xmax + step_size, step=step_size)
    bin_centres = np.arange(start=xmin + step_size / 2, stop=xmax + step_size / 2, step=step_size)
    
    print("bin_centres is:", bin_centres)

    # Convert awkward array to a numpy array for histogram
    data_x, _ = np.histogram(myy_np, bins=bin_edges)  # histogram the data

    print('data myy is:', myy_np)
    print("data_x is:", data_x)
    
    # Check for NaN values in histogram data
    if np.any(np.isnan(data_x)):
        raise ValueError("Histogram data contains NaN values!")

    data_x_errors = np.sqrt(data_x)  # statistical error on the data

    # Avoid fitting if all data_x is zero (which leads to NaNs)
    if np.all(data_x == 0):
        raise ValueError("All histogram counts are zero; cannot fit a model.")

    # Fit models
    polynomial_mod = PolynomialModel(4)  # 4th order polynomial
    gaussian_mod = GaussianModel()  # Gaussian

    # Set initial guesses for the parameters of the polynomial model
    pars = polynomial_mod.guess(data_x, x=bin_centres, c0=data_x.max(), c1=20000, c2=1000, c3=0, c4=0)
    print('data_x.max() is:', data_x.max())

    # Set initial gues


In [ ]:
data['myy'] #invriant masses

[Back to contents](#contents)

Call the function to plot the data

In [ ]:
plot_data(data)

[Back to contents](#contents)

<a id='going_further'></a>

## What can you do to explore this analysis?

* Increase the fraction of data used in '[Lumi, fraction, file path](#fraction)'
* Use data_B, data_C and data_D in '[Samples](#samples)'
* Check how many events are being thrown away by each cut in '[Applying a cut](#applying_cut)'
* Add more cuts from the [Higgs discovery paper](https://www.sciencedirect.com/science/article/pii/S037026931200857X#se0090) in '[Changing a cut](#changing_cut)' and '[Applying a cut](#applying_cut)'
* Find the reduced chi-squared for the fit in '[Plotting](#plotting)'
* Find the mean of the fitted Gaussian in '[Plotting](#plotting)'
* Find the width of the fitted Gaussian in '[Plotting](#plotting)'
* Try different initial guesses for the parameters of the fit in '[Plotting](#plotting)'
* Try different functions for the fit in '[Plotting](#plotting)'
* Your idea!

[Back to contents](#contents)

In [ ]:
data

In [ ]:
def plot_data(data):   
    # Define your range, binning, and calculate the histogram
    xmin = 100 # GeV
    xmax = 160 # GeV
    step_size = 2 # GeV
    
    bin_edges = np.arange(start=xmin, stop=xmax + step_size, step=step_size)
    bin_centres = np.arange(start=xmin + step_size/2, stop=xmax + step_size/2, step=step_size)

    data_x, _ = np.histogram(data['myy'], bins=bin_edges)
    data_x_errors = np.sqrt(data_x)  # Statistical error on the data

    # Define the polynomial and Gaussian models
    polynomial_mod = PolynomialModel(4)  # 4th order polynomial
    gaussian_mod = GaussianModel()

    # Initial parameter guesses
    pars = polynomial_mod.guess(data_x, x=bin_centres, c0=data_x.max(), c1=0, c2=0, c3=0, c4=0)
    pars += gaussian_mod.guess(data_x, x=bin_centres, amplitude=1600, center=125, sigma=2)

    # Combine the models
    model = polynomial_mod + gaussian_mod

    # Fit the model to the data
    out = model.fit(data_x, pars, x=bin_centres, weights=1/data_x_errors)

    # Check if fit was successful
    if out.success:
        print("Fit was successful!")
    else:
        print("Fit failed!")

    # Extract and print fit parameters
    params_dict = out.params.valuesdict()  # Get the fitted parameter values as a dictionary
    for param, value in params_dict.items():
        print(f"{param} = {value}")
    
    # Optional: return the parameters for further use
    return params_dict

    # ... (rest of your plotting code)


In [ ]:
plot_data(data)

In [ ]:
params, out = plot_data(data)
print("Fitted parameters: ", params)


In [ ]:
import matplotlib.pyplot as plt
import awkward as ak

# Assuming 'data_filtered' is your filtered DataFrame from the previous step
# First, flatten the 'photon_pt' lists into a 1D array using Awkward
all_photon_pt = ak.flatten(data['photon_e']).to_list()

# Now plot the distribution of photon transverse momenta
plt.figure(figsize=(10, 6))
plt.hist(all_photon_pt, bins=60, range = (100,160), color='b', alpha=0.7)

# Adding labels and title
plt.title('Distribution of Photon Energy', fontsize=14)
plt.xlabel('Photon_e [GeV]', fontsize=12)
plt.ylabel('Number of Events', fontsize=12)

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import awkward as ak

# Assuming 'data_filtered' is your filtered DataFrame from the previous step
# Flatten the 'photon_pt' lists into 1D arrays using Awkward
all_photon_pt1 = ak.flatten(data['photon_pt'][0]).to_list()
all_photon_pt2 = ak.flatten(data['photon_pt'][1]).to_list()

# Create a figure with two subplots (1 row, 2 columns)
plt.figure(figsize=(12, 6))

# Plot for the first photon
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
plt.hist(all_photon_pt1, bins=60, range=(0, 250), color='b', alpha=0.7)
plt.title('Photon 1: Transverse Momentum', fontsize=14)
plt.xlabel('Photon pT [GeV]', fontsize=12)
plt.ylabel('%', fontsize=12)

# Plot for the second photon
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
plt.hist(all_photon_pt2, bins=60, range=(0, 250), color='r', alpha=0.7)
plt.title('Photon 2: Transverse Momentum', fontsize=14)
plt.xlabel('Photon pT [GeV]', fontsize=12)
plt.ylabel('%', fontsize=12)

# Adjust the layout to make sure plots don't overlap
plt.tight_layout()

# Show the plots
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import awkward as ak

# Assuming 'data_filtered' is your filtered DataFrame from the previous step
# Flatten the 'photon_pt' lists into 1D arrays using Awkward
all_photon_pt1 = ak.flatten(data['photon_eta'][0]).to_list()
all_photon_pt2 = ak.flatten(data['photon_eta'][1]).to_list()

# Create a figure with two subplots (1 row, 2 columns)
plt.figure(figsize=(12, 6))

# Plot for the first photon
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
plt.hist(all_photon_pt1, bins=8, range=(-4, 4), color='b', alpha=0.7)
plt.title('Photon 1: Eta', fontsize=14)
plt.xlabel('Photon eta  ', fontsize=12)
plt.ylabel('%', fontsize=12)

# Plot for the second photon
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
plt.hist(all_photon_pt2, bins=8, range=(-4, 4), color='r', alpha=0.7)
plt.title('Photon 2: Eta', fontsize=14)
plt.xlabel('Photon eta  ]', fontsize=12)
plt.ylabel('%', fontsize=12)

# Adjust the layout to make sure plots don't overlap
plt.tight_layout()

# Show the plots
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import awkward as ak

# Assuming 'data_filtered' is your filtered DataFrame from the previous step
# Flatten the 'photon_pt' lists into 1D arrays using Awkward
all_photon_pt1 = ak.flatten(data['photon_ptcone20'][0]).to_list()
all_photon_pt2 = ak.flatten(data['photon_ptcone20'][1]).to_list()

# Create a figure with two subplots (1 row, 2 columns)
plt.figure(figsize=(12, 6))

# Plot for the first photon
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
plt.hist(all_photon_pt1, bins=10, range=(0, 10), color='b', alpha=0.7)
plt.title('Photon 1: ptcone20', fontsize=14)
plt.xlabel('ptcone20 value  ', fontsize=12)
plt.ylabel('%', fontsize=12)

# Plot for the second photon
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
plt.hist(all_photon_pt2, bins=10, range=(0, 10), color='r', alpha=0.7)
plt.title('Photon 2: ptcone20', fontsize=14)
plt.xlabel('ptcone20 value ', fontsize=12)
plt.ylabel('%', fontsize=12)

# Adjust the layout to make sure plots don't overlap
plt.tight_layout()

# Show the plots
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import awkward as ak

# Assuming 'data_filtered' is your filtered DataFrame from the previous step
# Flatten the 'photon_pt' lists into 1D arrays using Awkward
all_photon_pt1 = ak.flatten(data['photon_phi'][0]).to_list()
all_photon_pt2 = ak.flatten(data['photon_phi'][1]).to_list()

# Create a figure with two subplots (1 row, 2 columns)
plt.figure(figsize=(12, 6))

# Plot for the first photon
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first plot
plt.hist(all_photon_pt1, bins=10, range=(0, 10), color='b', alpha=0.7)
plt.title('Photon 1: phi', fontsize=14)
plt.xlabel('Photon phi  ', fontsize=12)
plt.ylabel('%', fontsize=12)

# Plot for the second photon
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second plot
plt.hist(all_photon_pt2, bins=10, range=(0, 10), color='r', alpha=0.7)
plt.title('Photon 2: phi', fontsize=14)
plt.xlabel('Photon phi  ]', fontsize=12)
plt.ylabel('%', fontsize=12)

# Adjust the layout to make sure plots don't overlap
plt.tight_layout()

# Show the plots
plt.show()


In [ ]:
data['photon_isTightID']